In [3]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

d:\llmenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [5]:
dataset['train'][0]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

In [6]:
example = dataset['train'][0]
dialogure_lines = example['dialogue'].split('\n')

for line in dialogure_lines:
    print(line)

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.


In [7]:
# Load the Model
model_name = 'google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name , torch_dtype = torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
#Example :

prompt = "Hello world"
tokens = tokenizer(prompt, return_tensors="pt") #pt is for pytorch

print(tokens["input_ids"])
print(tokenizer.decode(tokens["input_ids"][0],skip_special_tokens=True))

tensor([[8774,  296,    1]])
Hello world


In [9]:
# To check the trainable parameters
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0

    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\n All model parameters:{all_model_params} \n Percentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"
print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
 All model parameters:247577856 
 Percentage of trainable model parameters: 100.00%


# Test the Model with Zero Shot Inferencing

In [10]:
# Lets utilize this dialogue for inference
example = dataset['test'][200]
dialogure_lines = example['dialogue'].split('\n')

for line in dialogure_lines:
    print(line)

print(f"Summary of the Dialogure is: {dataset['test'][200]['summary']}")

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
Summary of the Dialogure is: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.


In [11]:
index = 200
dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f""""
Summarize the following conversation.
{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(inputs["input_ids"],
    max_new_tokens = 200,
    )[0],
    skip_special_tokens=True
)
dash_line = '-'.join('' for x in range(80))
print(dash_line)

print(f'Input Prompt: \n{prompt}')
print(dash_line)

print(f"BaseLine Human Summary: \n{summary}\n")
print(dash_line)

print(f"Model Generated - Zero Shot :\n{output}")

-------------------------------------------------------------------------------
Input Prompt: 
"
Summarize the following conversation.
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

-------------------------------------------------------------------------------
BaseLi

# Performance Full Fine Tuning

In [16]:
def tokenize_function(example):
    start_prompt = "Summarize the following conversation. \n\n"
    end_prompt = "\n\n Summary:"

    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, truncation = True, padding = "max_length", return_tensors='pt').input_ids
    example['labels'] = tokenizer(example['summary'], truncation = True, padding='max_length', return_tensors='pt').input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.ipynb
# The tokenize_function code is handling all data across all splits in batches.

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id','topic','dialogue','summary',])

In [18]:
# Now lets check the shapes of all three parts of the dataset but we will first take
# a subset to the data

tokenized_datasets = tokenized_datasets.filter(lambda example, index: index %100 ==0, with_indices=True)
print(f"Shapes of the datasets:")

print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Filter: 100%|██████████| 1500/1500 [00:00<00:00, 3110.87 examples/s]

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


# Fine Tune the Model with PreProcessed Dataset

In [19]:
# Now we will utilize the builtin Hugging Face Trainer Class

output_dir = f'./dialogue-summary-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir = output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)


trainer = Trainer (
    model = original_model,
    args = training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation']
)


max_steps is given, it will override any value given in num_train_epochs


In [20]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
1,47.750000


TrainOutput(global_step=1, training_loss=47.75, metrics={'train_runtime': 2027.8251, 'train_samples_per_second': 0.004, 'train_steps_per_second': 0.0, 'total_flos': 5478058819584.0, 'train_loss': 47.75, 'epoch': 0.0625})

In [21]:
# Save the model
trained_model_dir = "./trained_model"
trainer.save_model(trained_model_dir)

#load the saved model
trained_model = AutoModelForSeq2SeqLM.from_pretrained(trained_model_dir)